In [1]:
import pandas
import numpy as np
import scipy.optimize

# Police dataset
This dataset is from [https://data.police.uk/](https://data.police.uk/). Code for retrieving the dataset is given at the bottom.

In [83]:
import os
if os.path.exists('stop-and-search_202308.csv'):
    print("file already downloaded")
else:
    !wget "https://www.cl.cam.ac.uk/teaching/current/DataSci/data/stop-and-search_202308.csv"
police = pandas.read_csv('stop-and-search_202308.csv')

file already downloaded


## Basic tabulations

In [88]:
police.columns

Index(['force', 'month', 'Type', 'Date', 'Part of a policing operation',
       'Policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range',
       'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'Outcome', 'Outcome linked to object of search',
       'Removal of more than just outer clothing'],
      dtype='object')

In [87]:
print(police['Outcome'].value_counts())
print('Missing values:', np.sum(pandas.isna(police['Outcome'])))

A no further action disposal       1103890
Arrest                              192453
Community resolution                120487
Summons / charged by post            27649
Penalty Notice for Disorder          21579
Khat or Cannabis warning             14301
Caution (simple or conditional)       5478
Name: Outcome, dtype: int64
Missing values: 32463


In [89]:
print(police['Officer-defined ethnicity'].value_counts())
print('Missing values:', np.sum(pandas.isna(police['Officer-defined ethnicity'])))

White    893012
Black    277950
Asian    179884
Other     45571
Mixed      3566
Name: Officer-defined ethnicity, dtype: int64
Missing values: 118317


In [90]:
print(police['Gender'].value_counts())
print('Missing values:', np.sum(pandas.isna(police['Gender'])))

Male      1290022
Female     149925
Other        4134
Name: Gender, dtype: int64
Missing values: 74219


## Data preparation

In [105]:
# Let's treat outcome "A no further action disposal" as nothing found
# Prepare vectors y=1[something found], eth, and gender,
# and remove rows with missing values.

df = police[['Outcome', 'Officer-defined ethnicity', 'Gender']].dropna()
df['outcome'] = np.where(df['Outcome']=='A no further action disposal', 'nothing', 'found')

y = np.where(df['outcome']=='found', 1, 0)
eth = df['Officer-defined ethnicity']
gender = df['Gender']

# Define e and g to be integer-encoded versions of ethnicity and gender.
# (For the purposes of this analysis, it'll be more useful than one-hot encoding.)

ethnicity_levels = ['Asian','Black','Mixed','Other','White']
gender_levels = ['Female', 'Male', 'Other']
assert all(eth.isin(ethnicity_levels))
assert all(gender.isin(gender_levels))

In [94]:
x = df.groupby(['outcome','Gender','Officer-defined ethnicity']).apply(len).unstack(fill_value=0)
print(x)

Officer-defined ethnicity   Asian   Black  Mixed  Other   White
outcome Gender                                                 
found   Female               1806    4340    123    594   26168
        Male                43022   67734    896  10796  197688
        Other                  37      31      2     30     360
nothing Female               5001   11931    266   1561   88011
        Male               121629  190654   2224  31302  556743
        Other                 123     269      8    161    1085


## A naive linear model
Let $y$ be the indicator of whether something was found.
We'll use least squares to fit the simple model
$
y \approx \beta_\text{eth}
$.

In other words, we're using maximum likelihood estimation to fit
$
Y \sim N(\beta_\text{eth},\sigma^2).
$

In [109]:
import sklearn.linear_model
X = np.column_stack([np.where(eth==k,1,0) for k in ethnicity_levels])
m = sklearn.linear_model.LinearRegression(fit_intercept=False)
m.fit(X, y)
pandas.Series(m.coef_, index=ethnicity_levels)

Asian    0.261424
Black    0.262239
Mixed    0.290139
Other    0.256953
White    0.257703
dtype: float64

## Logistic regression

It's more sensible to model this binary response as a Binomial random variable, $Y\sim \operatorname{Bin}(1,p_\text{eth})$.

This is a constrained optimization, since each probability $p_\text{eth}$ must lie in the range $[0,1]$. 
(There's no constraint on the sum of the $p_\text{eth}$; each ethnic group can have its own $p_\text{eth}$ regardless of the other ethnic groups.)
To ensure this, let's use the parameterization
$$
p_\text{eth} =  \frac{e^{s_\text{eth}}}{1+e^{s_\text{eth}}}.
$$
The log likelihood for an individual observation is
$$
\log\operatorname{Pr}_Y(y) = \beta_\text{eth} 1_{y=1} - \log(1+e^{\beta_\text{eth}})
$$
and the log likelihood for the entire dataset is
$$
\log\operatorname{Pr}(y_1,\dots,y_n) = \sum_{i=1}^n \bigl[\theta_i 1_{y_i=1} - \log(1+e^{\theta_i})\bigr]
\qquad\text{where }
\theta_i = \beta_{\text{eth}_i}.
$$

In [130]:
ethnicity_code = {k:i for i,k in enumerate(ethnicity_levels)}
e = np.array([ethnicity_code[v] for v in eth])

def loglik(s):
    # This is a very silly implementation. The dataframe has 1.3 million rows, and there are only 5 classes,
    # so it'd be far more efficient to rewrite the log likelihood as a sum over 5 classes.
    θ = s[e]  # [gives θ1,...,θn]
    return np.sum(θ * np.where(y==1,1,0) - np.log(1+np.exp(θ)))

initial_guess = np.array([0 for _ in ethnicity_levels])
βhat = scipy.optimize.fmin(lambda s: -loglik(s), initial_guess, maxiter=1000)

phat = np.exp(βhat) / (1 + np.exp(βhat))
pandas.Series(phat, index=ethnicity_levels)

Optimization terminated successfully.
         Current function value: 780743.609856
         Iterations: 776
         Function evaluations: 1238


Asian    0.261425
Black    0.262239
Mixed    0.290130
Other    0.256951
White    0.257703
dtype: float64

Another way to achieve the same result is using `sklearn.linear_model.LogisticRegression`. This is for fitting the model
$$
\log\operatorname{Pr}(y_1,\dots,y_n) = \sum_{i=1}^n \bigl[\theta_i 1_{y_i=1} - \log(1+e^{\theta_i})\bigr]
$$
where the $\theta$ vector is a linear combination of feature vectors, similar to simple least-squares fitting:
$$
\theta_i = \beta_1 e_{1,i} + \dots + \beta_K e_{K,i}
$$

In [131]:
import sklearn.linear_model
X = np.column_stack([np.where(eth==k,1,0) for k in ethnicity_levels])
m = sklearn.linear_model.LogisticRegression(fit_intercept=False)
m.fit(X, y)

shat = m.coef_[0]
phat = np.exp(shat) / (1 + np.exp(shat))
pandas.Series(phat, index=ethnicity_levels)

Asian    0.261426
Black    0.262243
Mixed    0.290366
Other    0.256982
White    0.257704
dtype: float64

## A simple intersectional model to look at ethnicity and gender
We'll first use least-squares estimation to fit the model
$$
y \approx \alpha + \beta_\text{eth} + \gamma_\text{gender}.
$$
We'll then fit the model
$$
y \approx \delta_{\text{eth},\text{gender}}.
$$
The first model assumes that the effects of ethnicity and gender are additive. The second model
allows an arbitrary response for each ethnicity-gender pair.
The _difference_ between the two models thus tells us how much of the response is _not_ explained
by simple addition of effects.

In [205]:
X = np.column_stack([np.where(eth==k,1,0) for k in ethnicity_levels][1:] + [np.where(gender==k,1,0) for k in gender_levels][1:])
m0 = sklearn.linear_model.LinearRegression()
m0.fit(X, y)

df = pandas.DataFrame({'eth': eth, 'gender': gender}).drop_duplicates()
X = np.column_stack([np.where(df.eth==k,1,0) for k in ethnicity_levels][1:] + [np.where(df.gender==k,1,0) for k in gender_levels][1:])
df['pred'] = m0.predict(X)
res0 = df.set_index(['gender','eth'])['pred']

In [206]:
X = np.column_stack([np.where((eth==k1) & (gender==k2),1,0) for k1 in ethnicity_levels for k2 in gender_levels])
m1 = sklearn.linear_model.LinearRegression(fit_intercept=False)
m1.fit(X, y)

df = pandas.DataFrame({'eth': eth, 'gender': gender}).drop_duplicates()
X = np.column_stack([np.where((df.eth==k1) & (df.gender==k2),1,0) for k1 in ethnicity_levels for k2 in gender_levels])
df['pred'] = m1.predict(X)
res1 = df.set_index(['gender','eth'])['pred']

In [207]:
# Ethnicity and gender effects, from the additive model

K = len(ethnicity_levels)
print("ethnicity coefficients, compared to", ethnicity_levels[0])
print(pandas.Series(m0.coef_[:K-1], index=ethnicity_levels[1:]))
print()
print("gender coefficients, compared to", gender_levels[0])
print(pandas.Series(m0.coef_[K-1:], index=gender_levels[1:]))

ethnicity coefficients, compared to Asian
Black    0.001315
Mixed    0.030587
Other   -0.004103
White   -0.001378
dtype: float64

gender coefficients, compared to Female
Male     0.025243
Other   -0.017813
dtype: float64


In [209]:
# Intersectionality, i.e. difference between the full interaction model and the additive model.
# Entries with -ve values mean Prob(find) is lower than expected, i.e. police stop more innocent people,
# entries with +ve values mean Prob(find) is higher than expected, i.e. police stop fewer innocent people,
# relative to what we'd expect from their ethnicity and gender alone.

(res1 - res0).unstack() * 100

eth,Asian,Black,Mixed,Other,White
gender,,,,,
Female,2.809339,2.819563,4.838659,4.251973,-0.666020
Male,-0.117294,-0.163922,-0.587257,-0.191237,0.094841
Other,1.184167,-11.738986,-4.999538,-5.823686,3.110414


# Retrieving the dataset

In [37]:
import pandas
import re
import pathlib
import os

In [80]:
# Go to https://data.police.uk/data/
# Choose Custom Download: all forces, earliest date to latest date, stop-and-search data
# Download a zip file, and unzip it

ROOT_FOLDER = os.path.expanduser("~/winhome/Downloads/529b04592726f112e90ac22ff31caa31fd0081cc")

filenames = list(pathlib.Path(ROOT_FOLDER).rglob('*stop-and-search.csv'))
assert len(filenames) > 0, "No matching files. Did you download stop-and-search data?"

def get_dataframe(path):
    m = re.match(r'(?P<month>\d{4}-\d{2})-(?P<force>.+)-stop-and-search', path.stem)
    df = pandas.read_csv(path)
    df.insert(0, 'month', m.group('month'))
    df.insert(0, 'force', m.group('force'))
    return df
df = [get_dataframe(p) for p in filenames]

police = pandas.concat(df, axis=0, ignore_index=True, sort=False)

In [81]:
police.to_csv('stop-and-search_202308.csv', index=False)